In [1]:
# needed to install jinja2 openpyxl pandas tensorflow numpy

import pandas as pd
import numpy as np
import tensorflow as tf
import openpyxl
from openpyxl import load_workbook
from tqdm import tqdm

def main():
    

    # Unpersists graph from file
    with tf.gfile.FastGFile("output", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
    
    readFrom=0
    readTo=12
    
    #workbook = pd.ExcelFile("Copy_trial.xlsx")                ############################HT#################
    workbook = pd.ExcelFile("test.xlsx")
    print("workbook loaded")

    df=readRange(readFrom,readTo,workbook)

    Run(readFrom,readTo,df)



/home/intern_eyecare/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def readRange(fromRow,toRow,workbook):
    rows = workbook.book.sheet_by_index(0).nrows
    # subtract the number of rows to read from the total number of rows (and another 1 for the header)
    workbook_dataframe = pd.read_excel(workbook, skiprows = fromRow,skip_footer = rows-toRow-1)
    return workbook_dataframe

In [3]:
import re
def findit(string):
    b=re.search(".*\.jpg",string,re.IGNORECASE)
    c=re.search(".*\.jpeg",string,re.IGNORECASE)
    if b or c:
        return False
    else:
        return True

In [4]:
def addressConv(datarow):
    #pre written shared drive address
    address ="/"                              ############################HT#################
    #address="\\\\bosch.com\\DfsRB\\DfsIN\\LOC\\Kor\\BUD\\NBT\\EyeCare\\"
    # or forward slash addressing
    
    #address="//bosch.com/DfsRB/DfsIN/LOC/Kor/BUD/NBT/EyeCare/"
    i=0                                        ############################HT i=1#################
    while True:
        if not findit(datarow[i]):
            address = address+str(datarow[i])
            break
        #address = address+str(datarow[i])+"\\"
        address = address + str(datarow[i])+"/"
        i=i+1
    return address

In [5]:
############################### Inception #################################################
def classifier(address):
    # Read in the image_data
    image_data = tf.gfile.FastGFile(address, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line
                   in tf.gfile.GFile("./labels.txt")]

    
    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

        predictions = sess.run(softmax_tensor, \
                 {'DecodeJpeg/contents:0': image_data})

        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
            
        if(predictions[0][0]>predictions[0][1]):
            return False
        else:
            return True

In [14]:


def Run(start,end,df):
    book = load_workbook("./outputExcel/outfile.xlsx")
    writer = pd.ExcelWriter("./outputExcel/outfile.xlsx", engine='openpyxl') # Destination
    writer.book = book
    styled = None
    framelength=int(len(df.index))
    
    for framelength in tqdm(range(framelength,0,-1)):
        nextIndex=len(df.index)-framelength
        oneframe =df.iloc[nextIndex]
        address=addressConv(oneframe)
        oneframe=pd.DataFrame(oneframe)
        oneframe = oneframe.T
         #classifing using classifier trained on inception
        if classifier(address):
            
            styled = (oneframe.style.applymap(lambda v: 'background-color: %s' % 'yellow' if v!= None else ''))
            styled.to_excel(writer,"Sheet "+str(start)+" - "+str(end)+"  ",startrow=nextIndex ,index=False,header=None)
             #append_df_to_excel(writer,book, styled, header=None, index=False)
           
        else:
            #append_df_to_excel(writer,book, styled, header=None, index=False)
            oneframe.to_excel(writer,"Sheet "+str(start)+" - "+str(end)+"  ", startrow=nextIndex, index=False,header=None)
        writer.save()    
    
    print("Done saving")
#

In [15]:
if __name__=="__main__":
    main()

  0%|          | 0/12 [00:00<?, ?it/s]

workbook loaded


100%|██████████| 12/12 [00:15<00:00,  1.30s/it]

Done saving
